<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../../index.ipynb) / [MicroPython auf dem Microcontroller installieren](../index.ipynb) / ESP32 HelTec: 0,96'' Display (ssd1306)
***
<span style="font-size:20pt;">MicroPython auf dem ESP32 HelTec installieren: 0,96'' Display (ssd1306)</span>
***

# Flashen des ESP32 HelTec (beide Versionen)

Firmware lokal: `/030_InstallAndSetup/Esp32_NodeMCU/esp32-20220117-v1.18.bin`, oder online: [Micropython: Firmware ESP32](https://micropython.org/download/esp32/).

Vorbereitung:  
python -m pip install esptool

dann unter Windows (Port anpassen nicht vergessen):  
esptool.py --chip esp32 --port COM4 erase_flash  
esptool.py --chip esp32 --port COM4 --baud 460800 write_flash -z 0x1000 esp32-20220117-v1.18.bin

Linux:   
esptool.py --chip esp32 --port /dev/ttyUSB0 erase_flash  
esptool.py --chip esp32 --port /dev/ttyUSB0 --baud 460800 write_flash -z 0x1000 esp32-20220117-v1.18.bin
***

<!--https://www.az-delivery.de/products/esp32-developmentboard-->

# Initialisierung des ESP32 HelTec
## Informationen

1) Das Display des HelTec benötigt eine spezielle Sequenz beim Start um richtig zu funktionieren (Stand 2021-10-25).
```Python
pinReset = machine.Pin(16, machine.Pin.OUT)
pinReset.off()
time.sleep_ms(50)
pinReset.on()
```
Diese wird in die "/boot.py" geschrieben, damit sie jedesmal beim hochfahren ausgeführt wird


2) Wichtige Pins:
```Python
SDA: 4
SCL: 15
On board LED: 25
```


3) Der ssd1306-OLED-Display Treiber muss auf ESP32 Systeme (Stand 2021-10-25) geschrieben werden, also auch auf den HelTec.

[Source: mjrobot.org](https://mjrobot.org/micropython-on-esp-using-jupyter-notebook)


## Schreiben von `boot.py`, `main.py` und der Module `info`.py, `ssd1306`.py und `display`.py

In [4]:
# Mit dem Esp32 verbinden:
#%serialconnect --port=COM3 --baud=115200
%serialconnect


#========================================================================
# Write '/info.py'
# 
# TYPE='Esp32 HelTec'
#========================================================================

with open('/info.py', 'w') as f:
    f.write(r"""TYPE='Esp32 HelTec'""")

    
#========================================================================
# Write '/boot.py'
#========================================================================

#This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
#import webrepl
#webrepl.start()

with open('/boot.py', 'w') as f:
    f.write(r"""from machine import Pin
import time
p=Pin(16,Pin.OUT)
p.off()
time.sleep_ms(50)
p.on()""")

#========================================================================
# Write '/main.py'
#========================================================================

# Its important on some HelTec to give the I2C a Number: here 1
# i2c=I2C(1,scl=Pin(15),sda=Pin(4))

with open('/main.py', 'w') as f:
    f.write(r"""
from machine import Pin,I2C
import time
import ssd1306
try:
 i2c=I2C(1,scl=Pin(15),sda=Pin(4))
 d=ssd1306.SSD1306_I2C(128,64,i2c)
 d.fill(0)
 d.invert(0)
 d.text("Esp32 HelTec",17,23,1)
 d.text("OLED 128x64",20,37,1)
 d.show()
 for i in range(5):
  time.sleep_ms(200)
  d.invert(i%2)
 for i in range(45):
  d.scroll(0, -1)
  time.sleep_ms(10)
  d.show()
except:
 pass
finally:
 try: del d
 except: pass
 try: del i2c
 except: pass
 import gc
 gc.collect()""") 


#========================================================================
# Write '/ssd1306.py'
#========================================================================
    
with open('/ssd1306.py', 'w') as f:
    f.write(r"""from micropython import const
import framebuf

SET_CONTRAST = const(0x81)
SET_ENTIRE_ON = const(0xA4)
SET_NORM_INV = const(0xA6)
SET_DISP = const(0xAE)
SET_MEM_ADDR = const(0x20)
SET_COL_ADDR = const(0x21)
SET_PAGE_ADDR = const(0x22)
SET_DISP_START_LINE = const(0x40)
SET_SEG_REMAP = const(0xA0)
SET_MUX_RATIO = const(0xA8)
SET_IREF_SELECT = const(0xAD)
SET_COM_OUT_DIR = const(0xC0)
SET_DISP_OFFSET = const(0xD3)
SET_COM_PIN_CFG = const(0xDA)
SET_DISP_CLK_DIV = const(0xD5)
SET_PRECHARGE = const(0xD9)
SET_VCOM_DESEL = const(0xDB)
SET_CHARGE_PUMP = const(0x8D)

class SSD1306(framebuf.FrameBuffer):
 def __init__(self, width, height, external_vcc):
  self.width = width
  self.height = height
  self.external_vcc = external_vcc
  self.pages = self.height // 8
  self.buffer = bytearray(self.pages * self.width)
  super().__init__(self.buffer, self.width, self.height, framebuf.MONO_VLSB)
  self.init_display()

 def init_display(self):
  for cmd in (
   SET_DISP,
   SET_MEM_ADDR,
   0x00,
   SET_DISP_START_LINE,
   SET_SEG_REMAP | 0x01,
   SET_MUX_RATIO,
   self.height - 1,
   SET_COM_OUT_DIR | 0x08,
   SET_DISP_OFFSET,
   0x00,
   SET_COM_PIN_CFG,
   0x02 if self.width > 2 * self.height else 0x12,
   SET_DISP_CLK_DIV,
   0x80,
   SET_PRECHARGE,
   0x22 if self.external_vcc else 0xF1,
   SET_VCOM_DESEL,
   0x30,
   SET_CONTRAST,
   0xFF,
   SET_ENTIRE_ON,
   SET_NORM_INV,
   SET_IREF_SELECT,
   0x30,
   SET_CHARGE_PUMP,
   0x10 if self.external_vcc else 0x14,
   SET_DISP | 0x01
  ): self.write_cmd(cmd)
  self.fill(0)
  self.show()

 def poweroff(self):
  self.write_cmd(SET_DISP)

 def poweron(self):
  self.write_cmd(SET_DISP | 0x01)

 def contrast(self, contrast):
  self.write_cmd(SET_CONTRAST)
  self.write_cmd(contrast)

 def invert(self, invert):
  self.write_cmd(SET_NORM_INV | (invert & 1))

 def rotate(self, rotate):
  self.write_cmd(SET_COM_OUT_DIR | ((rotate & 1) << 3))
  self.write_cmd(SET_SEG_REMAP | (rotate & 1))

 def show(self):
  x0 = 0
  x1 = self.width - 1
  if self.width != 128:
   col_offset = (128 - self.width) // 2
   x0 += col_offset
   x1 += col_offset
  self.write_cmd(SET_COL_ADDR)
  self.write_cmd(x0)
  self.write_cmd(x1)
  self.write_cmd(SET_PAGE_ADDR)
  self.write_cmd(0)
  self.write_cmd(self.pages - 1)
  self.write_data(self.buffer)

class SSD1306_I2C(SSD1306):
 def __init__(self, width, height, i2c, addr=0x3C, external_vcc=False):
  self.i2c = i2c
  self.addr = addr
  self.temp = bytearray(2)
  self.write_list = [b"\x40", None]  # Co=0, D/C#=1
  super().__init__(width, height, external_vcc)

 def write_cmd(self, cmd):
  self.temp[0] = 0x80
  self.temp[1] = cmd
  self.i2c.writeto(self.addr, self.temp)

 def write_data(self, buf):
  self.write_list[1] = buf
  self.i2c.writevto(self.addr, self.write_list)""")

#========================================================================
# Write '/display.py'
#========================================================================
    
with open('/display.py', 'w') as f:
    f.write(r"""import ssd1306
import math
class Display(ssd1306.SSD1306_I2C):
 def __init__(self,i2c=None):
  import machine
  if i2c is None:
   i2c=machine.I2C(1,scl=machine.Pin(15),sda=machine.Pin(4))
  super().__init__(128,64,i2c)
  self._x=0
  self._y=0

 def setCenter(self,x,y):
  self._x=x
  self._y=y

 def lineP(self,x,y,alpha,r0,r1,color=1):
  c=math.cos(alpha)
  s=math.sin(alpha)
  super().line(
   int(c*r0+self._x+x),
   int(s*r0+self._y+y),
   int(c*r1+self._x+x),
   int(s*r1+self._y+y),
   color
  )

 def clear(self):
  super().fill(0)

 def circle(self,x,y,r,color=1):
  pX=self._x+x
  pY=self._y+y
  x=0
  y=r
  d=3-(2*r)
  while x<=y:
   super().pixel(pX+x,pY+y,color)
   super().pixel(pX+y,pY+x,color)
   super().pixel(pX-y,pY+x,color)
   super().pixel(pX-x,pY+y,color)
   super().pixel(pX-x,pY-y,color)
   super().pixel(pX-y,pY-x,color)
   super().pixel(pX+y,pY-x,color)
   super().pixel(pX+x,pY-y,color)
   x+=1
   if d < 0:
    d=d+(4*x)+6
   else:
    d=d+4*(x-y)+10
    y-=1

 def fillCircle(self,x,y,r,color=1):
  pX=self._x+x
  pY=self._y+y
  x=0
  y=r
  d=3-(2*r)
  while x<=y:
   super().hline(pX-x,pY+y,x+x+1,color)
   super().hline(pX-y,pY+x,y+y+1,color)
   super().hline(pX-x,pY-y,x+x+1,color)
   super().hline(pX-y,pY-x,y+y+1,color)
   x+=1
   if d < 0:
    d=d+(4*x)+6
   else:
    d=d+4*(x-y)+10
    y-=1

 def pixel(self,x,y,color=None):
  if color is None:
   return super().pixel(self._x+x,self._y+y)
  else:
   super().pixel(self._x+x,self._y+y,color)

 def hline(self,x,y,width,color=1):
  super().hline(self._x+x,self._y+y,width,color)

 def vline(self,x,y,height,color=1):
  super().vline(self._x+x,self._y+y,height,color)

 def line(self,x,y,x1,y1,color=1):
  super().line(self._x+x,self._y+y,self._x+x1,self._y+y1,color)

 def rect(self,x,y,width,height,color=1):
  super().rect(self._x+x,self._y+y,width,height,color)

 def fill_rect(self,x,y,width,height,color=1):
  super().fill_rect(self._x+x,self._y+y,width,height,color)

 def text(self,str,x,y,color=1):
  super().text(str,self._x+x,self._y+y,color)

 def blit(self,fbuf,x,y,key=0):
  super().blit(fbuf,self._x+x,self._y+y,key)""")

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


---
## [Check controller](../CheckController.ipynb)